## Riddle 1

In [18]:
from typing import List

with open('./input6') as input:
  fishes = [int(num) for num in input.readline().strip().split(',')]

def simulate_day(fishes: List[int]):
  for i in range(len(fishes)):
    if fishes[i] == 0:
      fishes.append(8)
      fishes[i] = 6
    else:
      fishes[i] -= 1

num_days = 80
for i in range(num_days):
  print(f'Simulating day {i}...')
  simulate_day(fishes)

print(f'After {num_days} days there are {len(fishes)} fish in the sea.')

Simulating day 0...
Simulating day 1...
Simulating day 2...
Simulating day 3...
Simulating day 4...
Simulating day 5...
Simulating day 6...
Simulating day 7...
Simulating day 8...
Simulating day 9...
Simulating day 10...
Simulating day 11...
Simulating day 12...
Simulating day 13...
Simulating day 14...
Simulating day 15...
Simulating day 16...
Simulating day 17...
Simulating day 18...
Simulating day 19...
Simulating day 20...
Simulating day 21...
Simulating day 22...
Simulating day 23...
Simulating day 24...
Simulating day 25...
Simulating day 26...
Simulating day 27...
Simulating day 28...
Simulating day 29...
Simulating day 30...
Simulating day 31...
Simulating day 32...
Simulating day 33...
Simulating day 34...
Simulating day 35...
Simulating day 36...
Simulating day 37...
Simulating day 38...
Simulating day 39...
Simulating day 40...
Simulating day 41...
Simulating day 42...
Simulating day 43...
Simulating day 44...
Simulating day 45...
Simulating day 46...
Simulating day 47...
Si

## Riddle 2

In [1]:
from typing import List
from numpy.typing import NDArray
import numpy as np

with open('./test_input6') as input:
  fishes = [int(num) for num in input.readline().strip().split(',')]

In [2]:


def numpy_append(arr: NDArray, value_arr: NDArray) -> NDArray:
  print(f'Current len: {len(arr)}')
  return_arr = arr
  insert_index = np.count_nonzero(return_arr != -2)
  while np.count_nonzero(return_arr != -2) + len(value_arr) > len(return_arr):
    return_arr = np.concatenate((return_arr, np.full(len(return_arr), -2, dtype='b')))
  return_arr[insert_index:insert_index+len(value_arr)] = value_arr
  print(f'New len: {len(return_arr)}')
  return return_arr

def simulate_days_numpy(fishes: List[int], num_days: int) -> List[int]:
  fish_np = np.array(fishes, dtype='b')
  for i in range(num_days):
    print(f'Simulating day {i}...')
    # Decrease all values by 1
    fish_np -= 1
    # Count -1s and add as many 8s (new fish)
    num_new = np.count_nonzero(fish_np == -1)
    fish_np = numpy_append(fish_np, np.full(num_new, 8, dtype='b'))
    print(f'Added {num_new} new fish.')
    # Set -1s to 6 (new cycle)
    fish_np[fish_np == -1] = 6
  return fish_np[fish_np != -2].tolist()


In [3]:

# Naiive solution (too slow)
def simulate_days_naiive(fishes: List[int], num_days: int) -> List[int]:
  new_fishes = list(fishes)
  for i in range(num_days):
    print(f'Simulating day {i}...')
    for i in range(len(new_fishes)):
      if new_fishes[i] == 0:
        new_fishes.append(8)
        new_fishes[i] = 6
      else:
        new_fishes[i] -= 1
  return new_fishes



num_days = 10
fishes = simulate_days_numpy(fishes, num_days)

print(f'After {num_days} days there are {len(fishes)} fish in the sea.')

Simulating day 0...
Current len: 5
New len: 5
Added 0 new fish.
Simulating day 1...
Current len: 5
New len: 10
Added 1 new fish.
Simulating day 2...
Current len: 10
New len: 20
Added 1 new fish.
Simulating day 3...
Current len: 20
New len: 40
Added 2 new fish.
Simulating day 4...
Current len: 40
New len: 80
Added 1 new fish.
Simulating day 5...
Current len: 80
New len: 80
Added 0 new fish.
Simulating day 6...
Current len: 80
New len: 80
Added 0 new fish.
Simulating day 7...
Current len: 80
New len: 80
Added 0 new fish.
Simulating day 8...
Current len: 80
New len: 160
Added 1 new fish.
Simulating day 9...
Current len: 160
New len: 320
Added 1 new fish.
After 10 days there are 161 fish in the sea.


## Clever approach

In [2]:
from typing import List

with open('./test_input6') as input:
  fishes = [int(num) for num in input.readline().strip().split(',')]

def add_descendants(days_newborn: List[int], start_idx: int, with_delay):
  i = start_idx + 1
  num_newborns = days_newborn[start_idx]
  if with_delay:
    while i < len(days_newborn):
      if (i - start_idx) == 8:
        days_newborn[i] += num_newborns
        break   
      i += 1

  start_idx = i - 1
  while i < len(days_newborn):
    if (i - start_idx) % 6 == 0:
      days_newborn[i] += num_newborns
    i += 1

test = [0,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
add_descendants(test, 0, True)
add_descendants(test, 1, True)
test

[0, 1, 1, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]

In [7]:

num_days = 80

days_cum = [0 for _ in range(num_days)]
days_newborn = [0 for _ in range(num_days)]

# Initialize
days_cum[0] = len(fishes)
for fish in fishes:
  days_newborn[fish] += 1

for fish in set(fishes):
  add_descendants(days_newborn, fish, False)

# Now do the actual simulation
for i in range(len(days_newborn)):
  add_descendants(days_newborn, i, True)

day = 1
while day < len(days_cum):
  days_cum[day] = days_cum[day-1] + days_newborn[day]
  day += 1

days_cum


[5,
 6,
 7,
 9,
 10,
 10,
 10,
 11,
 12,
 15,
 17,
 19,
 20,
 21,
 23,
 27,
 31,
 35,
 37,
 40,
 44,
 49,
 57,
 64,
 70,
 76,
 82,
 91,
 105,
 117,
 131,
 142,
 155,
 171,
 192,
 216,
 243,
 265,
 292,
 319,
 356,
 401,
 447,
 495,
 546,
 597,
 664,
 741,
 827,
 923,
 1015,
 1119,
 1237,
 1375,
 1536,
 1710,
 1890,
 2093,
 2303,
 2561,
 2850,
 3171,
 3522,
 3895,
 4297,
 4772,
 5291,
 5893,
 6546,
 7246,
 8022,
 8877,
 9846,
 10954,
 12158,
 13490,
 14941,
 16525,
 18343,
 20350]